<img src="https://drive.google.com/uc?export=view&id=15SNbs9etrW7wdww8q0BYpewEgOKC1d3m" width="100%" alt="Encabezado unidad 2"></img>

# Cassandra y Pandas
---

En este notebook mostraremos cómo leer y escribir datos en _Cassandra_ desde `pandas`.

Primero instalaremos e importaremos las librerías Necesarias:

In [ ]:
!pip install cassandra-driver

In [1]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
from IPython import display

Ahora, realizaremos la conexión a la base de datos de [astra](https://astra.datastax.com/) tal y cómo se mostró en el notebook `M3U2NB1_cassandra_python.ipynb`:

> **Nota**: recuerde subir los archivos `bundle.zip` y `token.json`.

In [2]:
cloud_config= {
        "secure_connect_bundle": "bundle.zip" # nombre de su archivo de bundle
}
with open('token.json') as user_file:
  token = json.loads(user_file.read())

In [3]:
auth_provider = PlainTextAuthProvider(
        token["clientId"],
        token["secret"]
        )
cluster = Cluster(
        cloud=cloud_config,
        auth_provider=auth_provider
        )
session = cluster.connect()

## **1. Escritura**
---

Primero, vamos a crear un conjunto de datos sintético en `pandas`

In [4]:
df = pd.DataFrame(
        data=np.random.uniform(size=(500, 4)),
        columns=[f"columna{i}" for i in range(1, 5)]
        ).reset_index().rename(columns={"index": "id"})

In [5]:
df.head()

,id,columna1,columna2,columna3,columna4
0,0,0.033428,0.726554,0.617573,0.301166
1,1,0.257129,0.518141,0.709545,0.545056
2,2,0.416254,0.604746,0.153769,0.993930
3,3,0.039410,0.850804,0.502700,0.761225
4,4,0.495070,0.471937,0.411307,0.636851


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        500 non-null    int64  
 1   columna1  500 non-null    float64
 2   columna2  500 non-null    float64
 3   columna3  500 non-null    float64
 4   columna4  500 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 19.7 KB


Ahora, veamos cómo escribir esta tabla dentro de _Cassandra_, primero creamos la tabla, debe reemplazar `<nombre_keyspace>` por el nombre de su KEYSPACE.

In [7]:
query = """
CREATE TABLE
    test.tabla_pandas (
        id INT PRIMARY KEY,
        columna1 FLOAT,
        columna2 FLOAT,
        columna3 FLOAT,
        columna4 FLOAT
        )
;
"""
session.execute(query)

Ahora insertamos los valores desde `pandas`, primero, preparamos los campos de la tabla que serán insertados:

In [8]:
query = """
INSERT INTO
    test.tabla_pandas (
            id, columna1, columna2,
            columna3, columna4
            )
VALUES
    (?, ?, ?, ?, ?)
;
"""
prepared = session.prepare(query)

Con esto, podemos insertar los valores desde `pandas`:

In [9]:
for row in df.itertuples():
    session.execute(
            prepared,
            (
                row.id, row.columna1, row.columna2,
                row.columna3, row.columna4
                )
            )

## **2. Lectura**
---

Ahora, veamos cómo podemos extraer una tabla de _Cassandra_ como un dataframe de `pandas`, para ello usaremos la siguiente consulta:

In [10]:
query = """
SELECT
    id,
    columna1 + columna2 AS columna5,
    columna3 + columna4 AS columna6
FROM
    test.tabla_pandas
;
"""

In [11]:
df = pd.DataFrame(session.execute(query).all())
df.head()

,id,columna5,columna6
0,23,1.443644,1.528181
1,114,1.194502,0.732621
2,53,0.729559,1.056051
3,110,1.325311,1.624600
4,91,0.816386,0.201507


Finalmente, eliminamos la tabla creada para este ejercicio:

In [12]:
query = """
DROP TABLE test.tabla_pandas;
"""
session.execute(query, timeout=None)

## **3. Recursos Adicionales**
---

* [Python read cassandra data into pandas](https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas).
* [How to insert pandas dataframe into cassandra(https://stackoverflow.com/questions/49108809/how-to-insert-pandas-dataframe-into-cassandra).

## **4. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*